In [21]:
import numpy             as np
import gvar              as gv
import matplotlib.pyplot as plt
import lsqfit
import itertools
from tqdm import tqdm
import copy

import corrfitter as cf

In [22]:
from b2heavy.FnalHISQMetadata import params as mData

In [23]:
from b2heavy.ThreePointFunctions.types3pts  import Ratio, RatioIO, ratio_prerequisites
from b2heavy.ThreePointFunctions.fitter3pts import RatioFitter, phys_energy_priors
from b2heavy.ThreePointFunctions.utils      import read_config_fit, dump_fit_object

In [24]:
from b2heavy.TwoPointFunctions.utils     import correlation_diagnostics
from b2heavy.TwoPointFunctions.types2pts import CorrelatorIO, plot_effective_coeffs
from b2heavy.TwoPointFunctions.fitter    import StagFitter

In [25]:
def tmax(y,errmax=0.3):
    for i,el in enumerate(y):
        if abs(gv.sdev(el)/gv.mean(el))>errmax:
            return i
    return len(y)

In [51]:
ENSEMBLE = 'Coarse-1'
MESON    = 'Dst'
MOMENTUM = '300'

DATA_DIR = '/Users/pietro/code/data_analysis/BtoD/Alex/'
DATA_2PT = '/Users/pietro/code/data_analysis/data/QCDNf2p1stag/B2heavy/lattice24'

TRANGE_EFF = (10,18) 

BINSIZE  = {
    'MediumCoarse':13,
    'Coarse-2':    16,
    'Coarse-1':    11,
    'Coarse-Phys': 19,
    'Fine-1':      16,
    'Fine-Phys':   16,
    'SuperFine':   22
}

NSTATES = 3

In [45]:
io = CorrelatorIO(ENSEMBLE,MESON,MOMENTUM,PathToDataDir=DATA_DIR)
stag = StagFitter(io, jkBin=BINSIZE[ENSEMBLE], smearing=['1S-1S','d-d','d-1S'])

# Second try (better but still crap)

In [28]:
xdata,ydata = stag.format(scale=True,shrink=True)

models = []
for smr,pol in ydata:
    if smr=='d-1S':
        continue
    sm1,sm2 = smr.split('-')
    fcn = cf.Corr2(
        datatag = (smr,pol),
        tp      = stag.Nt,
        s       = (1.,-1.),
        tdata   = xdata,
        tmin    = 15,
        tmax    = tmax(ydata[smr,pol]),
        dE      = ('dE','dE_o'),
        a       = (f'Z_{sm1}_{pol}',f'Z_{sm1}_{pol}_o'),
        b       = (f'Z_{sm2}_{pol}',f'Z_{sm2}_{pol}_o')
    )

    models.append(fcn)

In [29]:
meff, aeff = stag.meff((15,20))

e0  = gv.gvar(meff.mean,0.2)
de  = gv.gvar('0.5(5)')

z1s = np.sqrt(aeff['1S-1S','Unpol']); z1s = gv.gvar(z1s.mean,0.5) 
zd  = np.sqrt(aeff['d-d',  'Unpol']); zd  = gv.gvar(zd.mean ,0.5) 
z   = gv.gvar('1(5)')


priors = {
    'dE'             : [e0     ],
    'dE_o'           : [e0+de/2],
    'Z_1S_Unpol'     : [z1s],
    'Z_1S_Unpol_o'   : [z],
    'Z_d_Unpol'      : [zd],
    'Z_d_Unpol_o'    : [z],
    # 'Z_d-1S_Unpol'   : [z1s*zd,z,z],
    # 'Z_d-1S_Unpol_o' : [z1s*zd,z,z],
}

/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )


In [30]:
fitter = cf.CorrFitter(models=models)

fit = fitter.lsqfit(data=ydata,prior=priors)

In [31]:
print(fit)

Least Square Fit:
  chi2/dof [dof] = 1.2 [14]    Q = 0.28    logGBF = 312.46

Parameters:
   Z_1S_Unpol 0    1.09 (13)     [  0.88 (50) ]  
           dE 0   1.143 (16)     [  1.12 (20) ]  
 Z_1S_Unpol_o 0    0.52 (92)     [  1.0 (5.0) ]  
         dE_o 0    1.48 (23)     [  1.37 (32) ]  
    Z_d_Unpol 0   0.213 (26)     [  0.18 (50) ]  
  Z_d_Unpol_o 0    0.52 (92)     [  1.0 (5.0) ]  

Settings:
  svdcut/n = 1e-12/1    tol = (1e-08,1e-10,1e-10*)    (itns/time = 14/0.0)
  fitter = scipy_least_squares    method = trf



# GEVP with `EigenBasis` (nice, but not really)

In [32]:
xdata,ydata = stag.format(scale=True,shrink=True)

ydict = {}
for sm,pol in ydata:
    sm1,sm2 = sm.split('-')
    k = f'{sm1}-{sm2}_{pol}'
    ydict[k] = ydata[sm,pol]


for pol in stag.data.polarization.values:
    ydict[f'1S-d_{pol}'] = copy.deepcopy(ydict[f'd-1S_{pol}'])

In [33]:
xdata,ydata = stag.format(scale=True,shrink=True)

models = []
for smr,pol in ydata:
    sm1,sm2 = smr.split('-')
    fcn = cf.Corr2(
        datatag = (smr,pol),
        tp      = stag.Nt,
        s       = (1.,-1.),
        tdata   = xdata,
        tmin    = 15,
        tmax    = tmax(ydata[smr,pol]),
        dE      = ('m.dE','o.dE'),
        a       = (f'm.{sm1}',f'o.{sm1}'),
        b       = (f'm.{sm2}',f'o.{sm2}')
    )
    models.append(fcn)


fitter = cf.CorrFitter(models=models)

In [34]:
basis = cf.EigenBasis(
    ydict,
    keyfmt = '{s1}-{s2}_Unpol',
    srcs   = ['1S','d'],
    t      = (7,5) ,
    tdata  = xdata,
    osc    = True
)
prior = basis.make_prior(nterm=1, keyfmt='m.{s1}')

for k,p in prior.items():
    print(k,p)

m.1S [-1.63(49)]
m.d [-0.285(95)]
log(m.dE) [0.20(35)]
o.1S [-0.4(1.7)]
o.d [0.010(453)]
log(o.dE) [0.20(35)]


In [35]:
fit = fitter.lsqfit(data=ydata, prior=prior)
print(fit)

Least Square Fit:
  chi2/dof [dof] = 0.76 [22]    Q = 0.77    logGBF = 509.32

Parameters:
         m.1S 0    -1.10 (13)     [  -1.63 (49) ]  *
    log(m.dE) 0    0.138 (14)     [   0.20 (35) ]  
         o.1S 0    -1.3 (1.3)     [  -0.4 (1.7) ]  
    log(o.dE) 0     0.25 (10)     [   0.20 (35) ]  
          m.d 0   -0.221 (26)     [ -0.285 (95) ]  
          o.d 0    -0.14 (14)     [ 0.010 (453) ]  
-------------------------------------------------
         m.dE 0    1.148 (16)     [   1.22 (42) ]  
         o.dE 0     1.28 (13)     [   1.22 (42) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 14/0.0)
  fitter = scipy_least_squares    method = trf

